In [385]:
from pycaret import *
import pandas as pd

In [394]:
tmp = pd.read_csv('train/reg040_lstm_7_train.csv')
tmp2 = pd.read_csv('train/reg040_lstm_8_train.csv')
tmp3 = pd.read_csv('val/reg040_lstm_7_val.csv')
tmp4 = pd.read_csv('val/reg040_lstm_8_val.csv')
ls = [tmp,tmp2,tmp3,tmp4]

for i in ls:
    answer = np.argmax(i.drop('target',axis=1).values,axis=1)
    i['target'] = answer
tmp.to_csv('train/reg040_lstm_7_train.csv',index=False)
tmp2.to_csv('train/reg040_lstm_8_train.csv',index=False)
tmp3.to_csv('val/reg040_lstm_7_val.csv',index=False)
tmp4.to_csv('val/reg040_lstm_8_val.csv',index=False)

In [409]:
tmp1 = pd.read_csv('train/reg404_feature12_train_0.csv')
tmp2 = pd.read_csv('train/reg404_feature12_train_1.csv')
tmp3 = pd.read_csv('train/reg404_feature12_train_2.csv')
tmp4 = pd.read_csv('train/reg404_feature12_train_3.csv')
tmp5 = pd.read_csv('train/reg404_feature12_train_4.csv')
ls = [tmp1,tmp2,tmp3,tmp4,tmp5]
for i in ls:
    answer = np.argmax(i.values,axis=1)
    i['target'] = answer
tmp1.to_csv('train/reg404_feature12_train_0.csv',index=False)
tmp2.to_csv('train/reg404_feature12_train_1.csv',index=False)
tmp3.to_csv('train/reg404_feature12_train_2.csv',index=False)
tmp4.to_csv('train/reg404_feature12_train_3.csv',index=False)
tmp5.to_csv('train/reg404_feature12_train_4.csv',index=False)

In [502]:
train_ls = os.listdir('train')
valid_ls = os.listdir('val')
# train_ls.extend(valid_ls)

In [516]:
df = pd.read_csv('val/'+valid_ls[0])

In [524]:
for j in df:
    df[j] = 0
for i in valid_ls:
    data = pd.read_csv('val/'+i)
    if 'target' in data.columns:
        data.drop('target',axis=1,inplace = True)
    if sum(data.isnull().sum()) > 0:
        print(j)
    df += data
df = df/len(valid_ls)
df.drop('target',axis=1,inplace= True)

In [527]:
target = np.argmax(df.values,axis=1)

In [542]:
data1 = pd.read_csv('result/mean_ensemble_0130.csv')
data2 = pd.read_csv('result/mean_ensemble0204.csv')
data3 = pd.read_csv('result/mean_ensemble0203_del7')

FileNotFoundError: [Errno 2] No such file or directory: 'result/mean_ensemble0203_del7'

In [533]:
df2 = pd.read_csv('val/'+valid_ls[0])
for j in df2:
    df2[j] = 0
for i in valid_ls:
    data = pd.read_csv('val/'+i)
    if 'target' in data.columns:
        data.drop('target',axis=1,inplace = True)
    if sum(data.isnull().sum()) > 0:
        print(j)
    df2 += data
df2 = df2/len(valid_ls)
df2.drop('target',axis=1,inplace= True)

In [472]:
from tqdm.notebook import tqdm

In [473]:
df_list = []
for i in tqdm(train_ls):
    if 'train' not in i:
        data = pd.read_csv('val/'+i)
        if 'target' not in data.columns:
            print(i)
        df_list.append(pd.read_csv('val/'+i))
    else:
        data = pd.read_csv('train/'+i)
        if 'target' not in data.columns:
            print(i)
        df_list.append(pd.read_csv('train/'+i))
new = pd.concat(df_list)

  0%|          | 0/21 [00:00<?, ?it/s]

In [474]:
new.reset_index(drop=True,inplace=True)

In [475]:
new.shape

(79584, 112)

In [476]:
new['target'] = np.argmax(new.drop('target',axis=1).values,axis=1)

In [477]:
aug = os.listdir('train')[-5:]
aug_df = pd.read_csv('train/reg404_feature12_train_0.csv')
for i in aug_df:
    aug_df[i] = 0
for i in aug:
    data = pd.read_csv('train/'+i)
    aug_df += data
aug_df /= len(aug)

In [478]:
aug_df['target'] = np.argmax(aug_df.drop('target',axis=1).values,axis=1)

In [479]:
new = new.append(aug_df)

In [481]:
new.reset_index(drop=True,inplace = True)

In [469]:
from sklearn.model_selection import train_test_split
train,test = train_test_split(new,test_size = .2, random_state=42)
train_x = train.drop('target',axis=1)
train_y = train['target']
test_x = test.drop('target',axis=1)
test_y = test['target']
from sklearn.linear_model import LogisticRegression
rf = RandomForestClassifier()
rf.fit(train_x,train_y)
pred = rf.predict(test_x)
from sklearn.metrics import accuracy_score
accuracy_score(test_y,pred)

0.9818920105355575

In [482]:
from pycaret.classification import *
clf = setup(data = new, target = 'target')

,Description,Value
0,session_id,1401
1,Target,target
2,Target Type,Multiclass
3,Label Encoded,None
4,Original Data,"(85351, 112)"
5,Missing Values,0
6,Numeric Features,111
7,Categorical Features,0
8,Ordinal Features,0
9,High Cardinality Features,0


In [483]:
lr = create_model('lr',fold=5)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9847,0.0000,0.5322,0.9828,0.9833,0.9829,0.9829
1,0.9849,0.0000,0.5638,0.9830,0.9836,0.9832,0.9832
2,0.9855,0.0000,0.5567,0.9833,0.9840,0.9839,0.9839
3,0.9849,0.0000,0.5419,0.9839,0.9838,0.9831,0.9831
4,0.9859,0.0000,0.5419,0.9833,0.9843,0.9842,0.9842
Mean,0.9852,0.0000,0.5473,0.9832,0.9838,0.9835,0.9835
SD,0.0004,0.0000,0.0114,0.0003,0.0003,0.0005,0.0005


In [484]:
tune_lr = tune_model(lr,fold=5)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9850,0.0000,0.5515,0.9838,0.9841,0.9833,0.9833
1,0.9849,0.0000,0.5779,0.9836,0.9840,0.9832,0.9832
2,0.9850,0.0000,0.5896,0.9839,0.9842,0.9833,0.9833
3,0.9851,0.0000,0.5571,0.9844,0.9843,0.9834,0.9834
4,0.9863,0.0000,0.5521,0.9840,0.9848,0.9847,0.9847
Mean,0.9853,0.0000,0.5656,0.9839,0.9843,0.9836,0.9836
SD,0.0005,0.0000,0.0153,0.0003,0.0003,0.0006,0.0006


In [485]:
tune_lr

LogisticRegression(C=6.179, class_weight={}, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1401, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [486]:
log = LogisticRegression(C=6.179, class_weight={}, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1401, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [487]:
X = new.drop('target',axis=1)
y = new['target']

In [488]:
log.fit(X,y)

LogisticRegression(C=6.179, class_weight={}, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1401, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [489]:
test_ls = os.listdir('test')
df = pd.read_csv('test/reg040_lstm_2_test.csv')
for i in df:
    df[i] = 0
for j in test_ls:
    df += pd.read_csv('test/'+j)
df = df/len(test_ls)

In [500]:
arg = np.argmax(df.values,axis=1)
answer = list(map(lambda x: list(dic)[x],arg))

In [501]:
sub['label'] = answer
sub.to_csv('result/mean_ensemble0204.csv',index=False)

In [490]:
pred = log.predict(df)
answer = list(map(lambda x: list(dic)[x],pred))

In [492]:
sub['label'] = answer

In [494]:
sub.to_csv('result/ensemble0204.csv',index=False)

In [437]:
tmp1 = np.argmax(new.drop('target',axis=1).values,axis=1)
new['target'] = tmp1

In [445]:
from sklearn.model_selection import train_test_split
train,test = train_test_split(new,test_size = .2, random_state=42)
train_x = train.drop('target',axis=1)
train_y = train['target']
test_x = test.drop('target',axis=1)
test_y = test['target']
rf.fit(train_x,train_y)
pred = rf.predict(test_x)
from sklearn.metrics import f1_score
f1_score(test_y,pred,average='macro')

0.46783348795074314

In [ ]:
log.

In [446]:
accuracy_score(test_y,pred)

0.9801417608810263

In [449]:
sub['label']=answer

In [288]:
best_5 = compare_models(sort='F1',n_select=5,fold=5)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.9978,1.0000,0.9950,0.9978,0.9978,0.9975,0.9975,28.5000
gbc,Gradient Boosting Classifier,0.9973,0.9999,0.9909,0.9974,0.9973,0.9970,0.9970,595.5580
rf,Random Forest Classifier,0.9971,1.0000,0.9910,0.9971,0.9971,0.9967,0.9967,5.9780
lda,Linear Discriminant Analysis,0.9968,1.0000,0.9930,0.9969,0.9968,0.9964,0.9964,0.9680
knn,K Neighbors Classifier,0.9965,0.9996,0.9917,0.9965,0.9965,0.9960,0.9960,25.5560
svm,SVM - Linear Kernel,0.9964,0.0000,0.9910,0.9965,0.9964,0.9959,0.9959,0.9560
dt,Decision Tree Classifier,0.9960,0.9979,0.9912,0.9961,0.9960,0.9955,0.9955,0.9420
et,Extra Trees Classifier,0.9954,1.0000,0.9802,0.9955,0.9954,0.9948,0.9948,3.2080
ridge,Ridge Classifier,0.9932,0.0000,0.9707,0.9933,0.9931,0.9923,0.9923,0.1000
qda,Quadratic Discriminant Analysis,0.9817,0.9957,0.8192,0.9769,0.9780,0.9793,0.9793,0.6380


In [289]:
blended = blend_models(estimator_list = best_5, fold = 5, method = 'soft') # soft voting

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9985,1.0000,0.9942,0.9985,0.9984,0.9982,0.9982
1,0.9980,1.0000,0.9950,0.9980,0.9980,0.9977,0.9977
2,0.9985,1.0000,0.9970,0.9986,0.9985,0.9984,0.9984
3,0.9983,1.0000,0.9958,0.9983,0.9983,0.9980,0.9980
4,0.9979,1.0000,0.9960,0.9979,0.9979,0.9976,0.9976
Mean,0.9982,1.0000,0.9956,0.9982,0.9982,0.9980,0.9980
SD,0.0003,0.0000,0.0009,0.0003,0.0003,0.0003,0.0003


In [291]:
final_model = finalize_model(blended)

KeyboardInterrupt: 

In [ ]:
predictions = predict_model(final_model, data = test)

In [259]:
lr = create_model('lr')
tune_lr = tune_model(lr)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9971,1.0000,0.9905,0.9971,0.9971,0.9967,0.9967
1,0.9985,1.0000,0.9962,0.9985,0.9985,0.9982,0.9982
2,0.9975,1.0000,0.9959,0.9975,0.9975,0.9972,0.9972
3,0.9971,1.0000,0.9932,0.9971,0.9971,0.9967,0.9967
4,0.9983,1.0000,0.9974,0.9983,0.9983,0.9980,0.9980
5,0.9971,1.0000,0.9934,0.9971,0.9971,0.9967,0.9967
6,0.9973,1.0000,0.9937,0.9973,0.9973,0.9969,0.9969
7,0.9983,1.0000,0.9969,0.9983,0.9983,0.9980,0.9980
8,0.9981,1.0000,0.9960,0.9981,0.9981,0.9978,0.9978
9,0.9965,1.0000,0.9943,0.9966,0.9965,0.9961,0.9961


In [260]:
new_logit = LogisticRegression(**tune_lr.get_params())

In [262]:
X = new.drop('target',axis=1)
y = new['target']
new_logit.fit(X,y)

LogisticRegression(C=4.399, class_weight={}, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1133, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [271]:
new_pred = new_logit.predict(test)

In [266]:
test_ls = os.listdir('../feature12/')[:5]
for idx,i in enumerate(test_ls):
    globals()['data_{}'.format(idx)] = pd.read_csv('../feature12/'+i)

In [264]:
os.listdir('../feature12/')[:5]

['reg404_feature12_test_0.csv',
 'reg404_feature12_test_1.csv',
 'reg404_feature12_test_2.csv',
 'reg404_feature12_test_3.csv',
 'reg404_feature12_test_4.csv']

In [269]:
test = (test + data_0 + data_1 + data_2 + data_3 + data_4) / 6

In [263]:
test

,label 0,label 1,label 2,label 3,label 4,label 5,label 6,label 7,label 8,label 9,...,label 101,label 102,label 103,label 104,label 105,label 106,label 107,label 108,label 109,label 110
0,-1.308316,-1.939537,-1.931482,-1.812076,-1.945774,-1.925815,-1.930647,-1.932627,-1.945861,-1.854804,...,-1.907872,-1.782719,-2.142010,-1.825381,-1.298885,-1.955388,-1.940606,-1.888711,-2.019389,-1.985541
1,-1.679571,-2.123849,-2.030025,-2.011997,-1.996936,-2.058845,-2.105128,-2.004051,-2.006297,-2.020923,...,-2.136317,-2.411753,-2.401423,-1.990273,-2.337797,-2.053326,-2.083286,-1.169521,-2.086835,-2.068821
2,-2.181583,-2.053901,-1.945698,-2.063736,-2.044168,-1.998792,-2.002096,-1.993452,-2.069043,-2.004916,...,-2.033152,-2.153879,-2.248566,-2.007614,-1.758255,-2.066788,-2.437596,-2.177984,-2.012953,-2.011966
3,-2.118824,-2.115917,-2.161315,-2.123527,-2.079050,-2.133097,-2.148617,-2.156211,-2.119005,-2.130542,...,-2.138470,-2.203857,-2.267932,-2.124176,-2.229231,-2.097277,-2.084376,-2.404410,-2.095733,-2.120595
4,-2.119714,-2.048104,-2.012818,-2.022691,-2.006408,-2.060063,-2.084063,-2.026836,-2.030437,-2.032755,...,-2.059283,-2.477092,-3.191564,-2.028446,-1.595307,-2.091185,-2.073357,-2.576556,-2.108143,-2.024134
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51901,-1.969922,-1.998888,-2.012383,-2.053226,-2.061570,-2.024993,-2.069502,-2.018441,-2.027766,-2.014124,...,-2.034599,-2.078906,-2.020207,-2.041661,-2.332218,-2.062890,-2.138778,-1.849169,-2.064007,-2.066634
51902,-2.615346,-2.021090,-2.111871,-1.992804,-2.131541,-2.065113,-2.050995,-2.050909,-2.116966,-2.117320,...,-2.101076,-2.570630,-2.965819,-2.121115,-1.703344,-2.100160,-2.171137,-2.475230,-2.075259,-2.057465
51903,-2.791371,-2.106155,-2.075274,-2.075202,-2.177678,-2.147532,-2.133868,-2.137883,-2.093860,-2.073721,...,-2.039248,-2.459633,-3.191444,-2.117008,-2.801375,-2.157183,-2.287435,-1.458020,-2.160700,-2.205854
51904,-1.163695,-2.170496,-2.089987,-2.158168,-2.112575,-2.090415,-2.068102,-2.168033,-2.083511,-2.127357,...,-2.116558,-1.541406,-1.377027,-2.108537,-2.181774,-2.051134,-2.002232,-2.196365,-2.073268,-2.078985


In [221]:
test_ls = os.listdir('test')
for idx,i in enumerate(test_ls):
    globals()['data_{}'.format(idx)] = pd.read_csv('test/'+i)

In [222]:
len(test_ls)

7

In [296]:
data_0.shape

(51906, 111)

In [315]:
test_df = data_0.copy()
for i in test_df:
    test_df[i] = 0

In [316]:
test_ls = os.listdir('test')
# test_df = pd.DataFrame()
for i in test_ls:
    data = pd.read_csv('test/'+i)
    test_df += data

In [317]:
test_df = test_df/len(test_ls)

In [318]:
pred = new_logit.predict(test_df)

In [340]:
mean_pred = np.argmax(test_df.values,axis=1)
answer = []
for i in mean_pred:
    answer.append(list(dic)[i])
sub = pd.read_csv('sample_submission.csv')
sub['label'] = answer

In [341]:
sub.to_csv('result/mean_ensemble_02.csv',index=False)

In [334]:
answer = []
for i in pred:
    answer.append(list(dic)[i])
ensemble['label'] = answer

In [336]:
ensemble.to_csv('result/ensemble_0202.csv',index=False)

In [320]:
sub['label'] = new_pred
sub.to_csv('result/ensemble0202.csv',index=False)

In [328]:
ensemble['label'] = new_pred

In [331]:
ensemble.to_csv('result/ensemble_0202.csv',index=False)

In [223]:
test = (data_0 + data_1 + data_2 + data_3 + data_4 + data_5 +data_6)/len(test_ls)

In [224]:
dic = {'1_00_0': '딸기_정상',
 '1_a1_1': '딸기_딸기잿빛곰팡이병_초기',
 '1_a1_2': '딸기_딸기잿빛곰팡이병_중기',
 '1_a1_3': '딸기_딸기잿빛곰팡이병_말기',
 '1_a2_1': '딸기_딸기흰가루병_초기',
 '1_a2_2': '딸기_딸기흰가루병_중기',
 '1_a2_3': '딸기_딸기흰가루병_말기',
 '1_b1_1': '딸기_냉해피해_초기',
 '1_b1_2': '딸기_냉해피해_중기',
 '1_b1_3': '딸기_냉해피해_말기',
 '1_b6_1': '딸기_다량원소결핍 (N)_초기',
 '1_b6_2': '딸기_다량원소결핍 (N)_중기',
 '1_b6_3': '딸기_다량원소결핍 (N)_말기',
 '1_b7_1': '딸기_다량원소결핍 (P)_초기',
 '1_b7_2': '딸기_다량원소결핍 (P)_중기',
 '1_b7_3': '딸기_다량원소결핍 (P)_말기',
 '1_b8_1': '딸기_다량원소결핍 (K)_초기',
 '1_b8_2': '딸기_다량원소결핍 (K)_중기',
 '1_b8_3': '딸기_다량원소결핍 (K)_말기',
 '2_00_0': '토마토_정상',
 '2_a5_1': '토마토_토마토흰가루병_초기',
 '2_a5_2': '토마토_토마토흰가루병_중기',
 '2_a5_3': '토마토_토마토흰가루병_말기',
 '2_a6_1': '토마토_토마토잿빛곰팡이병_초기',
 '2_a6_2': '토마토_토마토잿빛곰팡이병_중기',
 '2_a6_3': '토마토_토마토잿빛곰팡이병_말기',
 '2_b2_1': '토마토_열과_초기',
 '2_b2_2': '토마토_열과_중기',
 '2_b2_3': '토마토_열과_말기',
 '2_b3_1': '토마토_칼슘결핍_초기',
 '2_b3_2': '토마토_칼슘결핍_중기',
 '2_b3_3': '토마토_칼슘결핍_말기',
 '2_b6_1': '토마토_다량원소결핍 (N)_초기',
 '2_b6_2': '토마토_다량원소결핍 (N)_중기',
 '2_b6_3': '토마토_다량원소결핍 (N)_말기',
 '2_b7_1': '토마토_다량원소결핍 (P)_초기',
 '2_b7_2': '토마토_다량원소결핍 (P)_중기',
 '2_b7_3': '토마토_다량원소결핍 (P)_말기',
 '2_b8_1': '토마토_다량원소결핍 (K)_초기',
 '2_b8_2': '토마토_다량원소결핍 (K)_중기',
 '2_b8_3': '토마토_다량원소결핍 (K)_말기',
 '3_00_0': '파프리카_정상',
 '3_a9_1': '파프리카_파프리카흰가루병_초기',
 '3_a9_2': '파프리카_파프리카흰가루병_중기',
 '3_a9_3': '파프리카_파프리카흰가루병_말기',
 '3_a10_1': '파프리카_파프리카잘록병_초기',
 '3_a10_2': '파프리카_파프리카잘록병_중기',
 '3_a10_3': '파프리카_파프리카잘록병_말기',
 '3_b3_1': '파프리카_칼슘결핍_초기',
 '3_b3_2': '파프리카_칼슘결핍_중기',
 '3_b3_3': '파프리카_칼슘결핍_말기',
 '3_b6_1': '파프리카_다량원소결핍 (N)_초기',
 '3_b6_2': '파프리카_다량원소결핍 (N)_중기',
 '3_b6_3': '파프리카_다량원소결핍 (N)_말기',
 '3_b7_1': '파프리카_다량원소결핍 (P)_초기',
 '3_b7_2': '파프리카_다량원소결핍 (P)_중기',
 '3_b7_3': '파프리카_다량원소결핍 (P)_말기',
 '3_b8_1': '파프리카_다량원소결핍 (K)_초기',
 '3_b8_2': '파프리카_다량원소결핍 (K)_중기',
 '3_b8_3': '파프리카_다량원소결핍 (K)_말기',
 '4_00_0': '오이_정상',
 '4_a3_1': '오이_오이노균병_초기',
 '4_a3_2': '오이_오이노균병_중기',
 '4_a3_3': '오이_오이노균병_말기',
 '4_a4_1': '오이_오이흰가루병_초기',
 '4_a4_2': '오이_오이흰가루병_중기',
 '4_a4_3': '오이_오이흰가루병_말기',
 '4_b1_1': '오이_냉해피해_초기',
 '4_b1_2': '오이_냉해피해_중기',
 '4_b1_3': '오이_냉해피해_말기',
 '4_b6_1': '오이_다량원소결핍 (N)_초기',
 '4_b6_2': '오이_다량원소결핍 (N)_중기',
 '4_b6_3': '오이_다량원소결핍 (N)_말기',
 '4_b7_1': '오이_다량원소결핍 (P)_초기',
 '4_b7_2': '오이_다량원소결핍 (P)_중기',
 '4_b7_3': '오이_다량원소결핍 (P)_말기',
 '4_b8_1': '오이_다량원소결핍 (K)_초기',
 '4_b8_2': '오이_다량원소결핍 (K)_중기',
 '4_b8_3': '오이_다량원소결핍 (K)_말기',
 '5_00_0': '고추_정상',
 '5_a7_1': '고추_고추탄저병_초기',
 '5_a7_2': '고추_고추탄저병_중기',
 '5_a7_3': '고추_고추탄저병_말기',
 '5_a8_1': '고추_고추흰가루병_초기',
 '5_a8_2': '고추_고추흰가루병_중기',
 '5_a8_3': '고추_고추흰가루병_말기',
 '5_b3_1': '고추_칼슘결핍_초기',
 '5_b3_2': '고추_칼슘결핍_중기',
 '5_b3_3': '고추_칼슘결핍_말기',
 '5_b6_1': '고추_다량원소결핍 (N)_초기',
 '5_b6_2': '고추_다량원소결핍 (N)_중기',
 '5_b6_3': '고추_다량원소결핍 (N)_말기',
 '5_b7_1': '고추_다량원소결핍 (P)_초기',
 '5_b7_2': '고추_다량원소결핍 (P)_중기',
 '5_b7_3': '고추_다량원소결핍 (P)_말기',
 '5_b8_1': '고추_다량원소결핍 (K)_초기',
 '5_b8_2': '고추_다량원소결핍 (K)_중기',
 '5_b8_3': '고추_다량원소결핍 (K)_말기',
 '6_00_0': '시설포도_정상',
 '6_a11_1': '시설포도_시설포도탄저병_초기',
 '6_a11_2': '시설포도_시설포도탄저병_중기',
 '6_a11_3': '시설포도_시설포도탄저병_말기',
 '6_a12_1': '시설포도_시설포도노균병_초기',
 '6_a12_2': '시설포도_시설포도노균병_중기',
 '6_a12_3': '시설포도_시설포도노균병_말기',
 '6_b4_1': '시설포도_일소피해_초기',
 '6_b4_2': '시설포도_일소피해_중기',
 '6_b4_3': '시설포도_일소피해_말기',
 '6_b5_1': '시설포도_축과병_초기',
 '6_b5_2': '시설포도_축과병_중기',
 '6_b5_3': '시설포도_축과병_말기'}

In [226]:
pred = new_logit.predict(test)

In [272]:
answer = []
for i in new_pred:
    answer.append(list(dic)[i])

In [273]:
sub = pd.read_csv('sample_submission.csv')
sub['label'] = answer

In [286]:
sub.to_csv('result/mean_ensemble_0130.csv',index=False)

In [280]:
sub['label'] = np.argmax(test.values,axis=1)

In [283]:
answer = []
for i in np.argmax(test.values,axis=1):
    answer.append(list(dic)[i])

In [285]:
sub['label'] = answer

In [238]:
answer = []
pred = np.argmax(test.values,axis=1)

In [240]:
for i in pred:
    answer.append(list(dic)[i])
sub['label'] = answer
sub.to_csv('result/mean_ensemble_0126.csv',index=False)

In [232]:
sub.to_csv('result/ensemble_0126.csv',index=False)

In [99]:
ens = pd.read_csv('ensemble.csv')

In [85]:
test_pred = log.predict(test)

In [ ]:
best_5 = compare_models(sort = 'F1', n_select = 5)

In [ ]:
blended = blend_models(estimator_list = best_5, fold = 5, method = 'hard') # soft voting

In [ ]:
final_model = finalize_model(blended)

In [ ]:
test = pd.read_csv('test_mean.csv')

In [ ]:
predictions = predict_model(final_model, data = test)